## This jupyter notebook was used to finetune the resnet model for guitar chords. 

In [2]:
import torch
import torchvision.models as models

resnet = models.resnet50(pretrained=True)


c:\Users\Sven\anaconda3\envs\ML2Project\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Sven\anaconda3\envs\ML2Project\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Sven/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


In [3]:
num_classes = 14

resnet.fc = torch.nn.Linear(resnet.fc.in_features, num_classes)

In [5]:
import torchvision.transforms as transforms

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to a uniform size
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the pixel values
])


In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class FretboardDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform

        # Get the list of image file paths
        self.image_paths = [os.path.join(self.image_folder, file) for file in os.listdir(self.image_folder)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image

# Create instances of the custom dataset class for the training and validation sets
train_dataset = FretboardDataset('path_to_train_folder', transform=transform)
validation_dataset = FretboardDataset('path_to_validation_folder', transform=transform)


In [4]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
